In [30]:
from pathlib import Path

import geopandas
import pandas
import snkit

from pyproj import Geod
from snail_datapkg.spatial import polygonise_targets

In [2]:
# set up variables for incoming and processed data paths
project_path = Path().resolve().parent # assume we're running from the project scripts directory, so get the parent
incoming_data_path = project_path / "incoming_data"
processed_data_path = project_path / "processed_data"

In [4]:
# read in the power lines
lines_df = geopandas.read_file(incoming_data_path / "starter-data-kit/data/ZAF/gridfinder/grid__ZAF.gpkg")

In [8]:
# Assume openstreetmap source lines are high-voltage, and inferred lines are medium
source_to_voltage = {
    "openstreetmap": "high",
    "gridfinder": "medium"
}
lines_df['line_voltage_class'] = lines_df.source.map(source_to_voltage)
lines_df

,source,geometry,line_voltage_class
0,gridfinder,"LINESTRING (22.41458 -22.12708, 22.41875 -22.1...",medium
1,gridfinder,"LINESTRING (29.63125 -22.12708, 29.63958 -22.1...",medium
2,openstreetmap,"LINESTRING (31.89792 -22.12708, 31.90208 -22.1...",high
3,openstreetmap,"LINESTRING (31.90625 -22.12708, 31.90208 -22.1...",high
4,openstreetmap,"LINESTRING (31.90625 -22.12708, 31.91042 -22.1...",high
...,...,...,...
47105,gridfinder,"LINESTRING (20.22292 -34.66042, 20.21875 -34.6...",medium
47106,gridfinder,"LINESTRING (19.51042 -34.66458, 19.52292 -34.6...",medium
47107,openstreetmap,"LINESTRING (20.04792 -34.80625, 20.03958 -34.7...",high
47108,gridfinder,"LINESTRING (20.05625 -34.81042, 20.05208 -34.8...",medium


In [22]:
power_df = pandas.read_excel(incoming_data_path / "global-energy-monitor" / "Global-Integrated-Power-March-2025.xlsx", sheet_name="Power facilities")

/home/mert2014/micromamba/envs/snail-datapkg/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [23]:
power_df.columns

Index(['Type', 'Country/area', 'Subregion', 'Region', 'Plant / Project name',
       'Unit / Phase name', 'Plant / Project name (local)',
       'Plant / Project name (other)', 'Capacity (MW)', 'Status', 'Start year',
       'Retired year', 'Technology', 'Fuel', 'Hydrogen production',
       'Hydrogen capable', 'Country/area 1 (hydropower only)',
       'Country/area 2 (hydropower only)',
       'Country/area 1 Capacity (MW) (hydropower only)',
       'Country/area 2 Capacity (MW) (hydropower only)', 'Owner', 'Parent',
       'CHP', 'CCS', 'Conversion/replacement', 'Unit conversion year',
       'Conversion from/replacement of (fuel)',
       'Conversion from/replacement of (GEM unit ID)', 'Captive Industry Type',
       'Captive Industry Use', 'Captive Non Industry Use', 'Latitude',
       'Longitude', 'Location accuracy', 'City', 'Local area (taluk, county)',
       'Major area (prefecture, district)',
       'Subnational unit (state, province)', 'GEM location ID',
       'GEM unit/p

In [27]:
to_rename = {
    'Type': 'generation_type',
    'Country/area': 'country',
    'Plant / Project name': 'name',
    'Capacity (MW)': 'capacity_mw',
    'Status': 'status',
    'Start year': 'start_year',
    'Technology': 'technology',
    'Latitude': 'lat',
    'Longitude': 'lng',
    'Location accuracy': 'location_accuracy',
    'City': 'city',
    'GEM location ID': 'gem_location_id',
    'GEM.Wiki URL': 'gem_wiki_url',
}
power_df = power_df.rename(columns=to_rename)[to_rename.values()].query("country == 'South Africa' and status == 'operating'")

geom = geopandas.points_from_xy(power_df.lng, power_df.lat, crs="EPSG:4326")
power_gdf = geopandas.GeoDataFrame(data=power_df, geometry=geom)
power_gdf

,generation_type,country,name,capacity_mw,status,start_year,technology,lat,lng,location_accuracy,city,gem_location_id,gem_wiki_url,geometry
3350,hydropower,South Africa,Drakensberg hydroelectric plant,1000.0,operating,1982,pumped storage,-28.5660,29.0834,exact,Okhahlamba Local Municipality,L603315,https://gem.wiki/Drakensberg_hydroelectric_plant,POINT (29.0834 -28.566)
3351,hydropower,South Africa,Gariep hydroelectric plant,360.0,operating,1971,conventional storage,-30.6233,25.5067,exact,Gariep Local Municipality,L603316,https://gem.wiki/Gariep_hydroelectric_plant,POINT (25.5067 -30.6233)
3352,hydropower,South Africa,Ingula hydroelectric plant,1332.0,operating,2016,pumped storage,-28.2809,29.5880,exact,Emnambithi/Ladysmith Local Municipality,L603317,https://gem.wiki/Ingula_hydroelectric_plant,POINT (29.588 -28.2809)
3353,hydropower,South Africa,Palmiet hydroelectric plant,400.0,operating,1988,pumped storage,-34.1985,18.9746,exact,Theewaterskloof Local Municipality,L603318,https://gem.wiki/Palmiet_hydroelectric_plant,POINT (18.9746 -34.1985)
3354,hydropower,South Africa,Steenbras hydroelectric plant,160.0,operating,1979,pumped storage,-34.1520,18.8982,exact,NaN,L603319,https://gem.wiki/Steenbras_hydroelectric_plant,POINT (18.8982 -34.152)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128216,solar,South Africa,Welkom Lejweleputswa District Municipality sol...,7.9,operating,2022,Assumed PV,-27.9220,26.6834,exact,Welkom,L100001021008,https://www.gem.wiki/Welkom_Lejweleputswa_Dist...,POINT (26.6834 -27.922)
128217,solar,South Africa,Welkom Lejweleputswa District Municipality sol...,10.8,operating,2023,Assumed PV,-27.9264,26.7080,exact,Welkom,L100001021009,https://www.gem.wiki/Welkom_Lejweleputswa_Dist...,POINT (26.708 -27.9264)
128218,solar,South Africa,Witzenberg Local Municipality Cape Winelands D...,6.3,operating,NaN,Assumed PV,-33.4324,19.2300,exact,Witzenberg Local Municipality,L100001020963,https://www.gem.wiki/Witzenberg_Local_Municipa...,POINT (19.23 -33.4324)
128219,solar,South Africa,Witzenberg Local Municipality Cape Winelands D...,2.0,operating,2020,Assumed PV,-33.2271,19.2364,exact,Witzenberg Local Municipality,L100001021004,https://www.gem.wiki/Witzenberg_Local_Municipa...,POINT (19.2364 -33.2271)


In [37]:
electrified_areas = polygonise_targets(incoming_data_path / "starter-data-kit/data/ZAF/gridfinder/targets__ZAF.tif", 4326)

In [40]:
electrified_areas.geometry = electrified_areas.centroid
electrified_areas

/tmp/ipykernel_1148458/2905210728.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  electrified_areas.geometry = electrified_areas.centroid


,area_km2,geometry,id
0,12308.778361,POINT (190.5 0.5),0
1,36933.884387,POINT (2510.5 0.5),1
2,98474.360659,POINT (1428.5 0.875),2
3,61490.916119,POINT (3158.9 2.3),3
4,24559.179798,POINT (1429.5 4),4
...,...,...,...
19280,NaN,POINT (900.72222 3008.61111),19280
19281,NaN,POINT (733 3010.5),19281
19282,NaN,POINT (856.65385 3039.88462),19282
19283,NaN,POINT (858.38889 3045.66667),19283
